# Télécharger des données sur Hendrix: description du problème

Nous avons souvent besoin d'accéder à des donner sur Hendrix dans le cadre de nos activités de recherche

1. Extraire des séries temporelles sur de larges domaines
2. Extraire des données en un point
3. Extraire un forecast entier

...

#### Ou sont stockées les données? Sous quels formats?

De nombreuses données sont disponibles sur le serveur de stockage Hendrix. Le format des données varie: parfois (rarement) les données sont au format ".nc" (netcdf), format bien souvent utilisé en recherche, parfois en ".grib", parfois en format ".fa" qui est un format maison à Météo-France.

#### Comment peut-on accéder à ces resources?

Des outils existent en python pour accéder à ces ressources:
    
    1. Vortex: pour accéder/télécharger aux resources sur Hendrix
    vortex.meteo.fr
    
    
    2. Epygram: pour ouvrir les fichiers, les traiter...etc
    https://opensource.umr-cnrm.fr/projects/epygram
    

**Donc on a une bibliothèque pour télécharger les fichiers et une bibliothèque pour les ouvrir et les traiter, on est bon non?**

En théorie oui, en pratique non. Voici les obstacles qui s'érigent sur notre route, lorsque par exemple, on cherche à télécharger les données d'AROME:

1. Généralement, on ne télécharge pas tout le domaine d'un modèle mais seulement une **zone géographique restreinte**: il faut utiliser une commande Epygram pour se placer sur son domaine. Il fut donc connaitre soit le nom du domaine, soit les indices dans le modèle qui délimitent ce domaine.


2. Si on travaille sur des forecast, nous avons **plusieurs manières de reconstruire des séries temporelles**: différentes dates d'analyses, différentes échances de forecast... Il faut faire des choix et les coder.


3. Pour accéder à une ressource sur Hendrix, il faut remplir la **"resource_description" de Vortex**. Il faut donc soit la connaitre, soit la deviner (plutôt long), soit la trouver quelque part.


4. Les variables ne sont pas forcément exprimées au format souhaité: certaines variables météo sont **spectrales**, il faut les convertir, d'autres comme les précip sont **cummulées** (cad qu'à chaque échéance, les précips correspondent à une somme de précip depuis la date d'analyse). Il faut donc prendre en compte toutes ces spécificités et les traiter.

On comprend donc qu'il n'y a pas de bouton magique sur Vortex ou Epygram qui télécharge ce que l'on veut. Il faut écrire un script.

#### Une fois que l'on a maitrise ces étapes, d'autres problèmes peuvent apparaitre

1. La "resource_description" de Vortex change (pour une même resource) dans le temps. Par exemple pour AROME, en Juillet 2017, la clef "namespace" de la "resource_description" a changé (oper.archive.fr => vortex.archive.fr).


2. Le nom des variables changent au cours du temps. Par exemple dans AROME, la composante zonale du vecteur rafale de vent change: 'CLSU.RAF60M.XFU' & 'CLSU.RAF.MOD.XFU'.


3. Les variables de surface d'un modèle ne sont pas stockées dans le même fichier que les variables atmosphériques: pour un même run il faut trouver la bonne variable dans le bon fichier.

#### Les extractions sont longues et Hendrix peut planter

Les extractions sont longues, et le serveur plante régulièrement. 

Par exemple, au début de l'année scolaire 2021, extraire une année d'AROME m'a pris plus d'un mois. En effet Hendrix a planté plusieurs fois, sxcen (où je stockais mes variables) a également planté (problème de stockage), et il fallait que j'aille tous les jours suivre l'extraction des variables: c'est une tâche fastidieuse.

# Quelles solutions nous proposons?

Avec Hugo, nous avons décidé de prendre le script qu'utilisait Isabelle (historiquement Vincent Vionnet, Louis Quéno puis Isabelle), et de l'adapter de manière générique afin de faciliter la gestion des extractions. 

Nous souhaitions un script:


1. dans lequel il est aisé de rajouter une variable à extraire


2. qui énumère clairement les "resources_description" de Vortex pour un modèle donné


3. qui gère les plantages d'Hendrix


4. qui nous préviens lorsqu'un problème est rencontré, ou au contraire lorsque tout va bien


5. qui facilite le prestagging


6. qui nous indique si des variables que nous téléchargons sont déjà disponible ou non au CEN


7. dans lequel nous pouvons facilement nous insérer pour coder les spécificités de notre projet

# Installation

### Installation d'extracthendrix

#### Télécharger le repo git

> git clone https://git.meteo.fr/cnrm-cen/louisletoumelin/extracthendrix

#### pip install

Pour les utilisateurs qui vont modifier le code d'extraction:


> pip install -e --user extracthendrix


Pour les simples utilisateurs

> pip install --user extracthendrix

### Installation de Vortex sur sxcen

Pour installer la version 1.8.0 de Vortex (par exemple), on se rend dans le fichier .bash_profile dans son home et on ajoute les lignes suivantes:

### Installation d'Epygram sur sxcen

# Utilisation

In [ ]:
import extracthendrix as eh
from datetime import datetime

#### Je ne connais rien sur Hendrix, sur AROME: où est la doc?

In [ ]:
eh.print_documentation()

#### Extraction de séries temporelle d'AROME

In [ ]:
import extracthendrix as eh
from datetime import datetime

config_user = dict(

#  Where you want to store the outputs
folder= '/cnrm/cen/users/NO_SAVE/letoumelinl/folder3/',

# Models are defined in the models.ini file
model_name = 'AROME',

# The domain can be defined by its name or its coordinates
# Existing domain can be found in the config_fa2nc.py file
domain = "alp",

# Variables to extract and to store in the netcdf file
# Variable are defined in the config_fa2nc.py file
variables_nc = ['snow_density', 'LWdown' ,'PSurf', 'Qair', 'Rainf', 'SCA_down', 'Snowf', 'Tair', 'Wind', 'Wind_DIR', 'Wind_Gust'],

# "local" if the FA file are on your computer or "hendrix" otherwise
getter = "hendrix",

# "timeseries" or "forecast".
# Ex: "forecast" = 1 analysis at 06:00 + 30 terms ahead
mode = "timeseries",

# For prestaging and sending mail during (your mail = destination) extraction
email_address = "louis.letoumelin@meteo.fr",

# datetime(year, month, day)
date_start = datetime(2020, 6, 1),
date_end = datetime(2020, 6, 30),

# Analysis hour
analysis_hour = 0,

# Delta between terms
delta_terms = 1,

# Term in hour after analysis
start_term = 6, # Default: 6
end_term = 6 + 23 ,# Defautl: 6+23 = 29

# How to group the netcdf files: "month", "year", "all"
group_by_output_file = "month"
)

In [ ]:
e = eh.Extractor(config_user)

In [ ]:
e.download()

#### Extraction de forecast d'AROME (ex: 1 anlyse + 30 termes)

In [ ]:
import extracthendrix as eh
from datetime import datetime

config_user = dict(

#  Where you want to store the outputs
folder= '/cnrm/cen/users/NO_SAVE/letoumelinl/folder3/',

# Models are defined in the models.ini file
model_name = 'AROME',

# The domain can be defined by its name or its coordinates
# Existing domain can be found in the config_fa2nc.py file
domain = "alp",

# Variables to extract and to store in the netcdf file
# Variable are defined in the config_fa2nc.py file
variables_nc = ['snow_density', 'LWdown' ,'PSurf', 'Qair', 'Rainf', 'SCA_down', 'Snowf', 'Tair', 'Wind', 'Wind_DIR', 'Wind_Gust'],

# "local" if the FA file are on your computer or "hendrix" otherwise
getter = "hendrix",

# "timeseries" or "forecast".
# Ex: "forecast" = 1 analysis at 06:00 + 30 terms ahead
mode = "forecast",

# For prestaging and sending mail during (your mail = destination) extraction
email_address = "louis.letoumelin@meteo.fr",

# datetime(year, month, day)
date_start = datetime(2020, 6, 1),
date_end = datetime(2020, 6, 30),

# Analysis hour
analysis_hour = 0,

# Delta between terms
delta_terms = 1,

# Term in hour after analysis
start_term = 6, # Default: 6
end_term = 6 + 23 ,# Defautl: 6+23 = 29

# How to group the netcdf files: "month", "year", "all"
group_by_output_file = "month"
)

#### Utilisation du prestagging

 <img src="http://confluence.meteo.fr/download/attachments/299881305/PutGetStage.jpg?version=1&modificationDate=1624261605335&api=v2" />

In [ ]:
e.prepare_prestaging_demand()

#### Gestion des plantages d'Hendrix

Si le code plante au moment d'accéder à Hendrix alors Hendrix bug probablement. 

Pour gérer ce problème, plutot que de soulever une erreur, nous avons décider de demander au système d'attendre qu'Hendrix reprenne ses forces pour continuer l'extraction: un petit plantage d'Hendrix ne changera donc rien à notre extraction.